# Week 3 Project. Segmenting and Clustering Toronto.
## Choosing the most suitable Neighbourhood.

### 1st. Importation of main libraries to the Jupiter Notebook.
We will make the imports in various cells as the speed for connection is not always available.

In [1]:
import pandas as pd
import numpy as np
import urllib.request

### 2nd. Importation of special libraries for visualization and coordinates

In [2]:
!conda install -c conda-forge geopy --yes #Needed to reinstall in order to work
from geopy.geocoders import Nominatim
import folium
import requests

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    nu

In [3]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### 3rd. Installation of BeatifulSoup.
#### this is a library that will help us to download easily the table within the Wikipedia article

In [4]:
conda install -c anaconda beautifulsoup4 #Needed to be installeed to make BeautifulSoup work

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    certifi-2019.9.11          |           py36_0         154 KB  anaconda
    beautifulsoup4-4.8.1       |           py36_0         153 KB  anaconda
    soupsieve-1.9.5            |           py36_0          61 KB  anaconda
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.5-py36_0      a

In [5]:
from bs4 import BeautifulSoup #import BeautifulSoup to read the table

### Now we will download the Wikipedia page with urllib library utilities.

In [6]:
webpageurl = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = urllib.request.urlopen(webpageurl)
CanadaZIPsWeb = req.read().decode()


### We are now using BeautifulSoup to extract the table.

In [7]:
CanadaZIPsoup = BeautifulSoup(CanadaZIPsWeb, 'html.parser')
CanadaZIPtable = CanadaZIPsoup.find('table', class_='sortable')


### Now we create a Panda Dataframe with the structure needed in order to append the data extracted with BeautifulSoup

In [8]:
dfCanadaNeigh=pd.DataFrame(columns=['Postcode','Borough','Neighbourhood'])
dfCanadaNeigh

,Postcode,Borough,Neighbourhood


### All data inside the table is after 'td' so we first create the list with all 'td'

In [9]:
ValuesString=CanadaZIPtable.find_all('td')
ValuesString

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td>, <td>M4A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
 </td>, <td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
 </td>, <td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Manor" title="Lawrence Manor">Lawrence Manor</a>
 </td>, <td>M7A</td>, <td>

#### Now we use a for in order to read the string and put it into the dataframe. We will add the data without the special characters.

In [10]:
for i in range(1,int(len(ValuesString)/3)): 
    dfCanadaNeigh=dfCanadaNeigh.append({'Postcode':ValuesString[3*i+0].text.strip('[]\n'),'Borough':ValuesString[3*i+1].text.strip('[]\n'),'Neighbourhood':ValuesString[3*i+2].text.strip('[]\n')},ignore_index=True)
dfCanadaNeigh.tail()

,Postcode,Borough,Neighbourhood
281,M8Z,Etobicoke,Mimico NW
282,M8Z,Etobicoke,The Queensway West
283,M8Z,Etobicoke,Royal York South West
284,M8Z,Etobicoke,South of Bloor
285,M9Z,Not assigned,Not assigned


##### We just perform a safety copy and generate a new one sorted and grouped.

In [11]:
dfCanadaNeighOriginal=dfCanadaNeigh

#### Now we join postcodes, neighbourhood as requested

In [12]:
index_cols = dfCanadaNeigh.columns.tolist()
index_cols.remove("Neighbourhood") 
dfCanadaNeigh = dfCanadaNeigh.groupby(index_cols)["Neighbourhood"].apply(list)
dfCanadaNeigh = dfCanadaNeigh.reset_index()


In [13]:
dfCanadaNeigh.tail()

,Postcode,Borough,Neighbourhood
174,M9V,Etobicoke,"[Albion Gardens, Beaumond Heights, Humbergate,..."
175,M9W,Etobicoke,[Northwest]
176,M9X,Not assigned,[Not assigned]
177,M9Y,Not assigned,[Not assigned]
178,M9Z,Not assigned,[Not assigned]


In [14]:
dfCanadaNeigh['Neighbourhood']=dfCanadaNeigh['Neighbourhood'].astype(str).str.replace(r"[\[\]']", '')
dfCanadaNeigh.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
dfCanadaNeigh

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
174,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
175,M9W,Etobicoke,Northwest
176,M9X,Not assigned,Not assigned
177,M9Y,Not assigned,Not assigned


#### For last, we will replace the "Not assigned", for the 'Borough' column in the same row.

In [16]:
dfCanadaNeighCopy=dfCanadaNeigh
dfCanadaNeigh['Neighbourhood'].replace("Not assigned", dfCanadaNeigh['Borough'], inplace=True)
dfCanadaNeigh.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
FinalCanadadf = dfCanadaNeigh[dfCanadaNeigh.Borough != 'Not assigned'] #this operation creates a new df without the Borough not yet assigned.
FinalCanadadf

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
169,M9N,York,Weston
170,M9P,Etobicoke,Westmount
171,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
174,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
